![Practicum AI Logo image](https://github.com/PracticumAI/practicumai.github.io/blob/main/images/logo/PracticumAI_logo_250x50.png?raw=true) <img src="https://github.com/PracticumAI/practicumai.github.io/blob/84b04be083ca02e5c7e92850f9afd391fc48ae2a/images/icons/practicumai_computer_vision.png?raw=true" alt="Practicum AI: Computer Vision icon" align="right" width=50>
***

# Computer Vision Concepts

You may recall *Practicum AI*"s heroine Amelia, the AI-savvy nutritionist. At the end of our *[Deep Learning Foundations course](https://practicumai.org/courses/deep_learning/)*, Amelia was helping with a computer vision project. Her colleague, an entomologist named Kevin, had a dataset of images of bees and wasps and wanted to classify them.

![Image of bees and wasps from the dataset cover image](https://github.com/PracticumAI/deep_learning/blob/main/images/bees_wasps_dataset-cover.png?raw=true)


## AI Pathway review for Bees vs Wasps

If you have taken our *[Getting Started with AI course]*(https://practicumai.org/courses/getting_started/), you may remember this figure of the **AI Application Development Pathway**. Let's take a quick review of how we applied this pathway in the case of the Bees vs Wasps example.

![AI Application Development Pathway image showing the 7 steps in developing an AI application](https://practicumai.org/getting_started/images/application_dev_pathway.png)

1. **Choose a problem to solve:** In this example, we need to classify images as bees, wasps, other insects, or a non-insect. 
2. **Gather data:** The data for the example come from [Kaggle](https://www.kaggle.com/datasets/jerzydziewierz/bee-vs-wasp), a great repository of datasets, code, and models.
3. **Clean and prepare the data:** In the *Deep Learning Foundations* course, we assumed that this was done for us. One issue that we ran into was that of class imbalance. There are many more images in some classes than others, leading to a poor performing model. We won't address this yet, but will come back to it in notebook [01.2_data_imbalance.ipynb](01.2_data_imbalance.ipynb).
4. **Choose a model:** In the *Deep Learning Foundations* course, we presented the model with little detail. Now that we know more about Convolutional Neural Networks (CNNs) and some other tools at our disposal, we will explore the model in more detail.
   * As part of the iterative process of training models, one thing we noticed is that most of our models were **overfitting** — performing better on the training data than they did on the testing data. Essentially, the models memorized the training data but did not generalize well to new data that had not been seen. 
      * In this notebook, we will explore **dropout** as one mechanism to mitigate overfitting.
5. **Train the model:** In training the model we may have had a few issues. With so many hyperparameters to tune, it's easy to lose track of what combinations have been tried and how changes impacted model performance. 
   * In this notebook, we introduce you to [TensorBoard](https://www.tensorflow.org/tensorboard), one popular tool in a class of tools known as **experiment tracking** or **MLOps (Machine learning operations) tools**. These tools help track changes to hyperparameters, the training process, and the data. They allow comparison among runs and can even automate multiple runs for you. Learning to use MLOps tools will help you as you continue to learn more about AI workflows.
6. **Evaluate the model:** We will continue to assess how the model performs on the validation set and adjust the model and hyperparameters to attempt to produce a better model. However, as noted above in step 3, one issue we will continue to encounter is the class imbalance.    
   * This is a common issue with real data, and in notebook [01.2_data_imbalance.ipynb](01.2_data_imbalance.ipynb), we will explore some methods to handle this.
7. **Deploy the model:** We won"t get to this stage in this exercise, but hopefully, we will end up with a model that could be deployed and achieve relatively good accuracy at solving the problem.


## A refresher

If you need a refresher, or haven"t taken the *Deep Learning Foundations* course, the final notebook is part of this repository: [DLF_03_bees_vs_wasps.ipynb](DLF_03_bees_vs_wasps.ipynb). No need to worry though, we will cover what we did before and the new changes as we work through this notebook. Some of the code has been updated to work better with the new things we add in this exercise and moved into the [helpers_01.py](helpers_01.py) file. It is imported below ratehr than included in the notebook to keep things cleaner.

## Outline of this notebook

This notebook covers a fair bit of ground. To orient you, here's an outline of the topics covered. Note that you can also open the notebook outline to see section headers.

1. Run through loading the data and exploring it a bit ([sections 1](#1.-Import-the-libraries-we-will-use) through 5).
1. Set initial hyperparameters, train a CNN model, and evaluate the performance ([sections 6](#6.-Make-our-model) through 8).
1. Explore Tensorboard as a tool to gain more insight into model performance ([section 9](#9.-View-training-metrics-in-TensorBoard)).
1. Spend some time tuning hyperparameters and evaluating the impact on model performance ([section 10](#10.-Explore-hyperparameters!))
1. Summarize the results obtaiend so far ([section 11](#11.-Summary-so-far))
1. Explore what a convolutional kernel is in more detail, visualizing kernels and convolved images ([section 12](#12.-A-look-inside-CNNs))
1. Add dropout to our model ([section 13](#13.-Dropout)) and discuss the padding and stride hyperparameters ([section 14](#14.-Padding-and-stride-for-convolutional-layers))
1. Conclude with a summary of the results and motivate the next notebook ([section 15](#15.-Conclusion))

## 1. Import the libraries we will use

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms.functional import to_pil_image
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np

# This has a lot of helper functions moved out of the main notebook to keep things clean
import helpers_01

In [10]:
# Print Pytorch versions and check for GPU
print(f'Pytorch version: {torch.__version__}')
print(f'  Should be "True" if Pytorch was built for GPU: {torch.cuda.is_available()}')
print(f'  Should list available GPU: {torch.cuda.get_device_name()}')


Pytorch version: 2.6.0+cu124
  Should be "True" if Pytorch was built for GPU: True
  Should list available GPU: NVIDIA GeForce RTX 2080 Ti


## 3. Getting the data

Check the "Getting the data" section of [DLF_03_bees_vs_wasps.ipynb](DLF_03_bees_vs_wasps.ipynb#2.-Getting-the-data) for details about the dataset and the code used to get the data if needed. Or you can look at the [helpers_01.py file](helpers_01.py). 

If you need to download the data, it is [hosted for public download from HiPerGator as a `tar.gz` file](https://data.rc.ufl.edu/pub/practicum-ai/Deep_Learning_Foundations/bee_vs_wasp.tar.gz). If you need to manually extract the data, you can add a cell and run: `helpers_01.extract_file("bee_vs_wasp.tar.gz", "data")`


In [11]:
# Check for the data.
# This will look for the data files required for this notebook in some common locations.
# If it can"t find the data, it will ask if you know where it is. If you do, answer yes and
# provide the path to the data (up to and including the `bee_vs_wasp` folder name). If not,
# it will ask if you want to download it. If you answer yes, it will download the data and
# extract it into your data folder.

data_path = helpers_01.manage_data()

Found data at data/bee_vs_wasp.


## 4. Examine some images

Many of the steps in this notebook are written as functions, making it easier to run these steps repeatedly as you work on optimizing the various hyperparameters.

The `helpers_01.load_display_data()` function takes: 
* A path to the data: set from above.
* The batch size: set as 32 below, but a good hyperparameter to tune.
* Target shape for images: set as 80x80 color images below, another possible hyperparameter.
* Whether or not to show sample images.

The function returns training and validation datasets. To help highlight the class imbalance issue, the function has been updated to report the numbers of images and percent of the total in each class.

In [ ]:
def load_display_data(
    path,
    batch_size=32,
    shape=(80, 80, 3),
    show_pictures=True,
    stratify=False,
    return_cls_counts=False,
):
    """Takes a path, batch size, target shape for images and optionally
    whether to show sample images. Returns training and validation datasets
    """
    print("******************************************************************")
    print("Load data:")
    print(f"  - Loading the dataset from: {path}.")
    print(f"  - Using a batch size of: {batch_size}.")
    print(f"  - Resizing input images to: {shape}.")
    print(f"  - Stratify when sampling? {stratify}")
    print(f"  - Returning class counts for later use? {return_cls_counts}")
    print("******************************************************************")

    # Define the image size using the 1st 2 elements of the shape parameter
    # We don't need the number of channels here, just the dimensions to use
    image_size = shape[:2]

    # Get the class names
    class_names = os.listdir(path)

    images = []  # Initialize the images list
    labels = []  # Initialize the labels list
    cls_counts = {}

    # Get the images and labels to use for training and validation
    for class_name in class_names:
        class_path = os.path.join(path, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            images.append(image_path)
            labels.append(class_name)

    # Print the number of number of images per class
    print("\nFor the full dataset: ")
    print("   Class          # of images     # of total")
    print("--------------------------------------------")
    for class_name in class_names:
        print(
            f"{class_name:>15} {labels.count(class_name):11}"
            f"         {labels.count(class_name)/len(labels)*100:.1f}%"
        )
        # Save class count to return if requested
        cls_counts[class_name] = labels.count(class_name)
    print("--------------------------------------------")
    
    # Define transformations
    mean = torch.tensor([0.485, 0.456, 0.406])
    std = torch.tensor([0.229, 0.224, 0.225])
    transform = transforms.Compose([
        transforms.Resize(shape[:2]),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

    if stratify: 
        print("Splitting the dataset with stratification, this may take a bit.")

        # Split the data using stratification
        image_train, image_val, labels_train, labels_val  = train_test_split(
            images, labels, test_size=0.2, stratify=labels, random_state=123
        )

        # Load the dataset using ImageFolder
        train_dataset = datasets.ImageFolder(path, transform=transform)

        # Create the dataloader
        data_train = DataLoader(image_train, labels_train)
        data_val = DataLoader(image_val, labels_val)

        # Get the class counts for the training data
        train_class_counts = []
        for class_name in class_names:
            train_class_counts.append(data_train.count(class_name))

        # Print the number of number of images per class
        print("\nFor the stratified split training dataset: ")
        print("   Class          # of images     # of total")
        print("--------------------------------------------")
 
        for class_name, count in zip(data_train.class_names, train_class_counts):
            print(
                 f"{class_name:>15} {count.numpy():11.0f}"
                 f"         {count.numpy()/len(train_labels)*100:.1f}%"
             )
            
    else:
        # Split the data randomly
        # Load the dataset using ImageFolder
        dataset = datasets.ImageFolder(path, transform=transform)

        # Calculate the sizes of training and validation sets
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size

        # Split the dataset into training and validation sets
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        # Create data loaders for training and validation sets
        data_train = DataLoader(train_dataset, batch_size=32, shuffle=True)
        data_val = DataLoader(val_dataset, batch_size=32, shuffle=False)
        

    if show_pictures:
        # Display the classes in the dataset
        print(f'The classes in your dataset are: {class_names}')

        # Display up to 3 images from each of the categories
        for class_name in class_names:
            plt.figure(figsize=(10, 10))

            # Get 3 random images for this class
            class_images = [img for img, label in zip(images, labels) if label == class_name]
            num_images = min(3, len(class_images))
            class_images = np.random.choice(class_images, num_images, replace=False)

            # Display the images
            for i, image_path in enumerate(class_images):
                image = Image.open(image_path)
                plt.subplot(1, num_images, i + 1)
                plt.imshow(image)
                plt.axis("off")
                plt.title(class_name)
            plt.show()

    if return_cls_counts:
        print(f"\nClass counts being returned: {cls_counts}.")
        return data_train, data_val, cls_counts

    return data_train, data_val


In [71]:
# The data path should be set from the cell above.
# If that failed and you want to set it manually, use the line below.
# data_path= "data/bee_vs_wasp"

data_train, data_valid = load_display_data(
    data_path, batch_size=32, shape=(80,80,3), stratify=True, show_pictures=True
)

******************************************************************
Load data:
  - Loading the dataset from: data/bee_vs_wasp.
  - Using a batch size of: 32.
  - Resizing input images to: (80, 80, 3).
  - Stratify when sampling? True
  - Returning class counts for later use? False
******************************************************************

For the full dataset: 
   Class          # of images     # of total
--------------------------------------------
           wasp        4943         43.3%
   other_insect        2439         21.4%
            bee        3183         27.9%
 other_noinsect         856         7.5%
--------------------------------------------
Splitting the dataset with stratification, this may take a bit.


ValueError: batch_size should be a positive integer value, but got batch_size=['other_insect', 'wasp', 'wasp', 'wasp', 'other_insect', 'wasp', 'wasp', 'bee', 'bee', 'other_insect', 'wasp', 'wasp', 'wasp', 'wasp', 'wasp', 'wasp', 'wasp', 'bee', 'wasp', 'other_insect', 'bee', 'other_insect', 'wasp', 'bee', 'wasp', 'other_insect', 'wasp', 'wasp', 'other_insect', 'bee', 'other_noinsect', 'wasp', 'wasp', 'wasp', 'bee', 'wasp', 'bee', 'other_insect', 'bee', 'bee', 'wasp', 'bee', 'other_insect', 'other_insect', 'other_insect', 'other_insect', 'wasp', 'wasp', 'other_insect', 'wasp', 'other_insect', 'wasp', 'wasp', 'other_insect', 'wasp', 'wasp', 'wasp', 'bee', 'other_noinsect', 'wasp', 'other_insect', 'bee', 'wasp', 'other_insect', 'wasp', 'wasp', 'other_noinsect', 'other_insect', 'wasp', 'wasp', 'wasp', 'bee', 'wasp', 'wasp', 'bee', 'wasp', 'other_insect', 'wasp', 'bee', 'wasp', 'wasp', 'other_insect', 'other_insect', 'wasp', 'wasp', 'other_insect', 'wasp', 'wasp', 'wasp', 'other_noinsect', '

In [6]:
# Define a custom dataset class
class BeeWaspDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_names = os.listdir(data_path)
        for idx, class_name in enumerate(self.class_names):
            class_path = os.path.join(data_path, class_name)
            for img_name in os.listdir(class_path):
                self.images.append(os.path.join(class_path, img_name))
                self.labels.append(idx)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((80, 80)),
    transforms.ToTensor()
])

# Load the dataset
data_path = 'data/bee_vs_wasp'
dataset = BeeWaspDataset(data_path, transform=transform)
train_data, val_data = train_test_split(dataset, test_size=0.2, stratify=dataset.labels)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)


In [7]:
# Create the model
model = helpers_01.make_model()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the model
model = helpers_01.compile_train_model(train_loader, val_loader, model, loss_fn, optimizer, num_epochs=10)


Epoch 1/10, Loss: 1.0587426123919186
Validation Loss: 0.9038829141192966
Epoch 2/10, Loss: 0.901039430103102
Validation Loss: 0.8573985339866744
Epoch 3/10, Loss: 0.852957592248083
Validation Loss: 0.8235902479953237
Epoch 4/10, Loss: 0.8184014229507713
Validation Loss: 0.7973514795303345
Epoch 5/10, Loss: 0.7862095437891834
Validation Loss: 0.7798186668919193
Epoch 6/10, Loss: 0.7582736819774121
Validation Loss: 0.7528111607664161
Epoch 7/10, Loss: 0.7313533175241697
Validation Loss: 0.7282624940077463
Epoch 8/10, Loss: 0.7133790619306631
Validation Loss: 0.7096356364587942
Epoch 9/10, Loss: 0.6985462966498796
Validation Loss: 0.7070517415801684
Epoch 10/10, Loss: 0.6718778260104306
Validation Loss: 0.6971876600550281


In [8]:
# Evaluate the model
helpers_01.evaluate_model(val_loader, model)


Accuracy: 73.08533916849015%
